In [2]:

import rtree, pygeos
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import os
import folium

import numpy as np
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans
import numpy as np
import statsmodels.api as sm
from datetime import datetime, timedelta

C:\Anaconda3\envs\geo37\lib\site-packages\geopandas\_compat.py:110: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.10.3-CAPI-1.16.1). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [3]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [4]:

data_path = 'C:/Users/mhudgell/data/dsa-jake/'
os.listdir(data_path)

['1 Team & Station Trip Level Data - April 2024 - Randomised Updated.csv',
 '1 Team & Station Trip Level Data - April 2024 - Randomised.csv',
 'Total Number of Vehicles in Fleet - April 2024.csv',
 'Workshop Data - April 2024 - Randomised.csv']

In [5]:
file_path = data_path +'1 Team & Station Trip Level Data - April 2024 - Randomised Updated.csv'
trip = pd.read_csv(file_path)


In [8]:
trip.head(2)

,Unnamed: 0,Registration,Fleet,Vehicle Function,Station,Date,Time - Start,Time - End,Trip Count,Miles Travelled,Hours,Speed,label,unavail_start,unavail_end,new_row,homeStationStart,homeStationEnd
0,0,111111,South Worcestershire,PATROL,WORCESTER,01-Apr-24,00:36:19,01:16:04,1,2.49,0.66,3.77,Unavailable,01/04/2024 00:36,01/04/2024 01:16,False,True,True
1,1,111111,South Worcestershire,PATROL,WORCESTER,01-Apr-24,02:23:22,02:30:37,1,2.30,0.12,19.17,Unavailable,01/04/2024 02:23,01/04/2024 02:30,False,True,True


In [9]:

sorted_df = trip.sort_values(by=['Date', 'Registration'], ascending=True)


In [10]:
len(sorted_df.index)

5194

In [267]:

midnight_df = sorted_df[(sorted_df['unavail_end'] < sorted_df['unavail_start'])]

com_df['new_row'] = False
new_row_list = []
new_endtime = '23:59:59'
for idx, row in sorted_df.iterrows():
    if sorted_df.at[idx,'unavail_end'] < sorted_df.at[idx,'unavail_start']:
        new_row_copy = row.copy(deep=True)
        new_row_list.append(new_row_copy)
        sorted_df.at[idx, 'Time - End'] = new_endtime
        


In [268]:
print(len(new_row_list))
print(len(midnight_df.index))

70
70


In [269]:
new_df = pd.DataFrame(new_row_list)

In [270]:
new_df['Time - Start'] = '00:00:01'
new_df['new_row'] = True

In [271]:
new_df['Date'] = pd.to_datetime(new_df['Date']) + timedelta(days=1)
new_df['Date'] = new_df['Date'].dt.strftime('%Y-%m-%d')

In [272]:
new_df['Time - End'] = new_df['unavail_end'].dt.time

new_df['Time - End'] = new_df['Time - End'].astype(str)


In [273]:
new_df['unavail_start']  = pd.to_datetime(new_df['Date'] + ' ' + new_df['Time - Start']) 
new_df['unavail_end']  = pd.to_datetime(new_df['Date'] + ' ' + new_df['Time - End']) 


In [274]:
sorted_df = sorted_df.append(new_df)
sorted_df['homeStationStart'] = True
sorted_df['homeStationEnd'] = True

In [275]:
sorted_df

,Registration,Fleet,Vehicle Function,Station,Date,Time - Start,Time - End,Trip Count,Miles Travelled,Hours,Speed,label,unavail_start,unavail_end,new_row,homeStationStart,homeStationEnd
0,111111,South Worcestershire,PATROL,WORCESTER,01-Apr-24,00:36:19,01:16:04,1,2.49,0.66,3.77,Unavailable,2024-04-01 00:36:19,2024-04-01 01:16:04,False,True,True
1,111111,South Worcestershire,PATROL,WORCESTER,01-Apr-24,02:23:22,02:30:37,1,2.30,0.12,19.17,Unavailable,2024-04-01 02:23:22,2024-04-01 02:30:37,False,True,True
2,111111,South Worcestershire,PATROL,WORCESTER,01-Apr-24,03:24:11,03:32:05,1,2.24,0.13,17.23,Unavailable,2024-04-01 03:24:11,2024-04-01 03:32:05,False,True,True
3,111111,South Worcestershire,PATROL,WORCESTER,01-Apr-24,03:48:21,03:54:24,1,1.93,0.10,19.30,Unavailable,2024-04-01 03:48:21,2024-04-01 03:54:24,False,True,True
4,111111,South Worcestershire,PATROL,WORCESTER,01-Apr-24,03:59:36,05:46:48,1,37.97,1.79,21.21,Unavailable,2024-04-01 03:59:36,2024-04-01 05:46:48,False,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2801,AAAAAA,South Worcestershire,PATROL,WORCESTER,2024-04-28,00:00:01,00:18:42,1,3.92,0.38,10.32,Unavailable,2024-04-28 00:00:01,2024-04-28 00:18:42,True,True,True
5088,FFFFFF,South Worcestershire,PATROL,WORCESTER,2024-04-28,00:00:01,00:20:51,1,15.54,0.86,18.07,Unavailable,2024-04-28 00:00:01,2024-04-28 00:20:51,True,True,True
1104,444444,South Worcestershire,PATROL,WORCESTER,2024-04-29,00:00:01,00:26:56,1,14.29,0.48,29.77,Unavailable,2024-04-29 00:00:01,2024-04-29 00:26:56,True,True,True
1818,666666,South Worcestershire,PATROL,WORCESTER,2024-04-29,00:00:01,00:00:37,1,1.24,0.13,9.54,Unavailable,2024-04-29 00:00:01,2024-04-29 00:00:37,True,True,True


In [276]:
file_path = data_path +'1 Team & Station Trip Level Data - April 2024 - Randomised Updated.csv'
sorted_df.to_csv(file_path)
